In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import subprocess
from functools import reduce

In [2]:
from ufs2arco import sources

In [3]:
gfs = sources.RDAGFSArchive(
    t0={"start": "2015-12-31T00", "end": "2024-12-31T00", "freq": "1YE"},
    fhr={"start": 0, "end": 6, "step": 6},
)

In [66]:
def pull_both_local(dims, cache_dir):
    a = gfs._open_local(
        dims=dims,
        file_suffix="",
        cache_dir="./gribcache",
    )
    b = gfs._open_local(
        dims=dims,
        file_suffix="b",
        cache_dir="./gribcache",
    )
    return a,b 

In [162]:
def open_datasets(dims, cache_dir, **kwargs):

    dsdict = {}
    for file_suffix in ["", "b"]:
        try:
            dsdict[file_suffix] = gfs.open_grib(
                dims=dims,
                file_suffix=file_suffix,
                cache_dir="./gribcache",
                **kwargs,
            ) 
        except:
            dsdict[file_suffix] = None

    if all([xx is not None for xx in dsdict.values()]):
        alist = set(sorted(dsdict[""].data_vars))
        blist = set(sorted(dsdict["b"].data_vars))
        xds = xr.merge(list(dsdict.values()))
        
    elif dsdict[""] is not None or dsdict["b"] is not None:
        if dsdict[""] is not None:
            alist = set(sorted(dsdict[""].data_vars))
            blist = set()
            xds = dsdict[""]
        else:
            alist = set()
            blist = set(sorted(dsdict["b"].data_vars))
            xds = dsdict["b"]
    else:
        raise

    varlist = set(sorted(xds.data_vars))
    onlyA = alist - varlist.intersection(blist)
    onlyB = blist - varlist.intersection(alist)
    for key in varlist:
        if key in onlyA:
            xds[key].attrs["file_suffix"] = [""]
        elif key in onlyB:
            xds[key].attrs["file_suffix"] = ["b"]
        else:
            xds[key].attrs["file_suffix"] = ["", "b"]
    return xds

### First, figure out surface stepTypes available

In [68]:
dsdict = {}
for t0 in gfs.t0:
    dsdict[t0] = {}

    for fhr in gfs.fhr:
        surface_step_types = []
        for file_suffix in ["", "b"]:
            print(f"Reading (t0, fhr) = ({str(t0)}, {int(fhr)})")
            a = gfs._open_local(
                dims={"t0": t0, "fhr": fhr},
                file_suffix=file_suffix,
                cache_dir="./gribcache",
            )
            output = subprocess.check_output(
                ["grib_ls", "-p", "typeOfLevel,stepType", a],
                stderr=subprocess.DEVNULL
            ).decode()
    
            for line in output.splitlines():
                parts = line.strip().split()
                if len(parts) >= 2:
                    type_of_level, step_type = parts[-2], parts[-1]
                    if type_of_level == "surface":
                        surface_step_types.append(step_type)
        dsdict[t0][fhr] = sorted(set(surface_step_types))

Reading (t0, fhr) = (2015-12-31 00:00:00, 0)
Reading (t0, fhr) = (2015-12-31 00:00:00, 0)
Reading (t0, fhr) = (2015-12-31 00:00:00, 6)
Reading (t0, fhr) = (2015-12-31 00:00:00, 6)
Reading (t0, fhr) = (2016-12-31 00:00:00, 0)
Reading (t0, fhr) = (2016-12-31 00:00:00, 0)
Reading (t0, fhr) = (2016-12-31 00:00:00, 6)
Reading (t0, fhr) = (2016-12-31 00:00:00, 6)
Reading (t0, fhr) = (2017-12-31 00:00:00, 0)
Reading (t0, fhr) = (2017-12-31 00:00:00, 0)
Reading (t0, fhr) = (2017-12-31 00:00:00, 6)
Reading (t0, fhr) = (2017-12-31 00:00:00, 6)
Reading (t0, fhr) = (2018-12-31 00:00:00, 0)
Reading (t0, fhr) = (2018-12-31 00:00:00, 0)
Reading (t0, fhr) = (2018-12-31 00:00:00, 6)
Reading (t0, fhr) = (2018-12-31 00:00:00, 6)
Reading (t0, fhr) = (2019-12-31 00:00:00, 0)
Reading (t0, fhr) = (2019-12-31 00:00:00, 0)
Reading (t0, fhr) = (2019-12-31 00:00:00, 6)
Reading (t0, fhr) = (2019-12-31 00:00:00, 6)
Reading (t0, fhr) = (2020-12-31 00:00:00, 0)
Reading (t0, fhr) = (2020-12-31 00:00:00, 0)
Reading (t

In [69]:
for t0, fdict in dsdict.items():
    print(f"t0 = {t0}")
    print(f"\t{fdict[0]} \t {fdict[6]}")

t0 = 2015-12-31 00:00:00
	['instant'] 	 ['accum', 'avg', 'instant']
t0 = 2016-12-31 00:00:00
	['instant'] 	 ['accum', 'avg', 'instant']
t0 = 2017-12-31 00:00:00
	['instant'] 	 ['accum', 'avg', 'instant']
t0 = 2018-12-31 00:00:00
	['instant'] 	 ['accum', 'avg', 'instant']
t0 = 2019-12-31 00:00:00
	['instant'] 	 ['accum', 'avg', 'instant']
t0 = 2020-12-31 00:00:00
	['instant'] 	 ['accum', 'avg', 'instant']
t0 = 2021-12-31 00:00:00
	['instant'] 	 ['accum', 'avg', 'instant']
t0 = 2022-12-31 00:00:00
	['instant'] 	 ['accum', 'avg', 'instant']
t0 = 2023-12-31 00:00:00
	['instant'] 	 ['accum', 'avg', 'instant']
t0 = 2024-12-31 00:00:00
	['instant'] 	 ['accum', 'avg', 'instant']


* analysis: instant
* forecast: instant, accum, avg

### Now, get the variables

In [70]:
for stepType in ["instant", "accum", "avg"]:
    vdict[stepType] = {}
    for t0 in gfs.t0:
        vdict[stepType][t0] = {}
        dslist = []
        varlist = []
        for fhr in gfs.fhr:
            if fhr == 0 and stepType != "instant":
                vdict[stepType][t0][fhr] = set(xds.data_vars)
            else:
                xds = open_datasets(
                    dims={"t0": t0, "fhr": fhr},
                    cache_dir="./gribcache",
                    filter_by_keys={
                        "typeOfLevel": "surface",
                        "stepType": stepType,
                    },
                )
                vdict[stepType][t0][fhr] = set(xds.data_vars)

In [71]:
vdict["accum"]

{Timestamp('2015-12-31 00:00:00'): {np.int64(0): {'SUNSD',
   'cape',
   'cfrzr',
   'cicep',
   'cin',
   'cnwat',
   'cpofp',
   'cpr',
   'crain',
   'csnow',
   'fldcp',
   'fricv',
   'fsr',
   'gust',
   'hindex',
   'lftx',
   'lftx4',
   'lsm',
   'orog',
   'prate',
   'sde',
   'sdwe',
   'siconc',
   'sit',
   'sithick',
   'slt',
   'sp',
   't',
   'unknown',
   'veg',
   'vis',
   'wilt'},
  np.int64(6): {'acpcp', 'tp', 'watr'}},
 Timestamp('2016-12-31 00:00:00'): {np.int64(0): {'acpcp', 'tp', 'watr'},
  np.int64(6): {'acpcp', 'tp', 'watr'}},
 Timestamp('2017-12-31 00:00:00'): {np.int64(0): {'acpcp', 'tp', 'watr'},
  np.int64(6): {'acpcp', 'tp', 'watr'}},
 Timestamp('2018-12-31 00:00:00'): {np.int64(0): {'acpcp', 'tp', 'watr'},
  np.int64(6): {'acpcp', 'tp', 'watr'}},
 Timestamp('2019-12-31 00:00:00'): {np.int64(0): {'acpcp', 'tp', 'watr'},
  np.int64(6): {'acpcp', 'tp', 'watr'}},
 Timestamp('2020-12-31 00:00:00'): {np.int64(0): {'acpcp', 'tp', 'watr'},
  np.int64(6): {'a

In [72]:
vdict["instant"]

{Timestamp('2015-12-31 00:00:00'): {np.int64(0): {'SUNSD',
   'cape',
   'cin',
   'cnwat',
   'cpofp',
   'fldcp',
   'gust',
   'hindex',
   'lftx',
   'lftx4',
   'lsm',
   'orog',
   'sde',
   'sdwe',
   'siconc',
   'sithick',
   'sp',
   't',
   'unknown',
   'wilt'},
  np.int64(6): {'SUNSD',
   'cape',
   'cin',
   'cnwat',
   'cpofp',
   'fldcp',
   'gust',
   'hindex',
   'lftx',
   'lftx4',
   'lsm',
   'orog',
   'sde',
   'sdwe',
   'siconc',
   'sithick',
   'sp',
   't',
   'unknown',
   'wilt'}},
 Timestamp('2016-12-31 00:00:00'): {np.int64(0): {'SUNSD',
   'cape',
   'cin',
   'cnwat',
   'cpofp',
   'fldcp',
   'gust',
   'hindex',
   'lftx',
   'lftx4',
   'lsm',
   'orog',
   'sde',
   'sdwe',
   'siconc',
   'sithick',
   'sp',
   't',
   'unknown',
   'wilt'},
  np.int64(6): {'SUNSD',
   'cape',
   'cin',
   'cnwat',
   'cpofp',
   'fldcp',
   'gust',
   'hindex',
   'lftx',
   'lftx4',
   'lsm',
   'orog',
   'sde',
   'sdwe',
   'siconc',
   'sithick',
   'sp',
 

In [75]:
vdict["avg"]

{Timestamp('2015-12-31 00:00:00'): {np.int64(0): {'acpcp', 'tp', 'watr'},
  np.int64(6): {'avg_al',
   'avg_ishf',
   'avg_slhtf',
   'avg_utaua',
   'avg_vtaua',
   'cduvb',
   'cfrzr',
   'cicep',
   'cpr',
   'crain',
   'csnow',
   'duvb',
   'gflux',
   'iegwss',
   'ingwss',
   'prate',
   'sdlwrf',
   'sdswrf',
   'sulwrf',
   'suswrf'}},
 Timestamp('2016-12-31 00:00:00'): {np.int64(0): {'avg_al',
   'avg_ishf',
   'avg_slhtf',
   'avg_utaua',
   'avg_vtaua',
   'cduvb',
   'cfrzr',
   'cicep',
   'cpr',
   'crain',
   'csnow',
   'duvb',
   'gflux',
   'iegwss',
   'ingwss',
   'prate',
   'sdlwrf',
   'sdswrf',
   'sulwrf',
   'suswrf'},
  np.int64(6): {'avg_al',
   'avg_ishf',
   'avg_slhtf',
   'avg_utaua',
   'avg_vtaua',
   'cduvb',
   'cfrzr',
   'cicep',
   'cpr',
   'crain',
   'csnow',
   'duvb',
   'gflux',
   'iegwss',
   'ingwss',
   'prate',
   'sdlwrf',
   'sdswrf',
   'sulwrf',
   'suswrf'}},
 Timestamp('2017-12-31 00:00:00'): {np.int64(0): {'avg_al',
   'avg_ish

In [76]:
for stepType, d2 in vdict.items():
    for t0, d3 in d2.items():
        intersect = reduce(set.intersection, [set(x) for x in d3.values()]) 
        if len(d3[0] - intersect) > 0:
            print(f"More in analysis t0 = {t0}, stepType = {stepType}")
        if len(d3[6] - intersect) > 0:
            print(f"More in forecast t0 = {t0}, stepType = {stepType}")

More in forecast t0 = 2019-12-31 00:00:00, stepType = instant
More in forecast t0 = 2020-12-31 00:00:00, stepType = instant
More in forecast t0 = 2021-12-31 00:00:00, stepType = instant
More in forecast t0 = 2022-12-31 00:00:00, stepType = instant
More in forecast t0 = 2023-12-31 00:00:00, stepType = instant
More in forecast t0 = 2024-12-31 00:00:00, stepType = instant
More in analysis t0 = 2015-12-31 00:00:00, stepType = accum
More in forecast t0 = 2015-12-31 00:00:00, stepType = accum
More in analysis t0 = 2015-12-31 00:00:00, stepType = avg
More in forecast t0 = 2015-12-31 00:00:00, stepType = avg


### Get the common variables in each

In [77]:
intersect_analysis = {
    key: sorted(reduce(set.intersection, [set(x[0]) for x in vdict[key].values()]))
    for key in vdict.keys()
}

In [78]:
intersect_analysis

{'instant': ['SUNSD',
  'cape',
  'cin',
  'cnwat',
  'cpofp',
  'fldcp',
  'gust',
  'hindex',
  'lftx',
  'lftx4',
  'lsm',
  'orog',
  'sde',
  'sdwe',
  'siconc',
  'sithick',
  'sp',
  't',
  'unknown',
  'wilt'],
 'accum': [],
 'avg': []}

In [79]:
intersect_forecast = {
    key: sorted(reduce(set.intersection, [set(x[6]) for x in vdict[key].values()]))
    for key in vdict.keys()
}

In [80]:
intersect_forecast

{'instant': ['SUNSD',
  'cape',
  'cin',
  'cnwat',
  'cpofp',
  'fldcp',
  'gust',
  'hindex',
  'lftx',
  'lftx4',
  'lsm',
  'orog',
  'sde',
  'sdwe',
  'siconc',
  'sithick',
  'sp',
  't',
  'unknown',
  'wilt'],
 'accum': ['acpcp', 'tp', 'watr'],
 'avg': ['avg_al',
  'avg_ishf',
  'avg_slhtf',
  'avg_utaua',
  'avg_vtaua',
  'cduvb',
  'cfrzr',
  'cicep',
  'cpr',
  'crain',
  'csnow',
  'duvb',
  'gflux',
  'iegwss',
  'ingwss',
  'prate',
  'sdlwrf',
  'sdswrf',
  'sulwrf',
  'suswrf']}

In [81]:
for key in intersect_forecast.keys():
    print(f" --- {key} --- ")
    print("analysis - forecast: ", set(intersect_analysis[key]) - set(intersect_forecast[key]))
    print("forecast - analysis: ",set(intersect_forecast[key]) - set(intersect_analysis[key]))
    print()

 --- instant --- 
analysis - forecast:  set()
forecast - analysis:  set()

 --- accum --- 
analysis - forecast:  set()
forecast - analysis:  {'acpcp', 'watr', 'tp'}

 --- avg --- 
analysis - forecast:  set()
forecast - analysis:  {'avg_utaua', 'suswrf', 'duvb', 'gflux', 'iegwss', 'cicep', 'avg_slhtf', 'avg_ishf', 'sdlwrf', 'avg_vtaua', 'cfrzr', 'avg_al', 'prate', 'sdswrf', 'cpr', 'cduvb', 'csnow', 'crain', 'ingwss', 'sulwrf'}



In [83]:
intersect = {
    0: intersect_analysis,
    6: intersect_forecast,
}

So... same variables in `instant` forecast and analysis, otherwise all surface variables are in the forecast

### Get the unique per t0 variables

In [84]:
for stepType, d2 in vdict.items():
    print(f"stepType = {stepType}")
    for t0, d3 in d2.items():
        for fhr, d4 in d3.items():
            unique = set(d4) - set(intersect[fhr][stepType])
            if len(unique) > 0:
                print(f"\t{t0}, fhr = {fhr}")
                print(f"\t\t{unique}")

stepType = instant
	2017-12-31 00:00:00, fhr = 0
		{'landn', 'vis'}
	2017-12-31 00:00:00, fhr = 6
		{'landn', 'vis'}
	2018-12-31 00:00:00, fhr = 0
		{'landn', 'vis'}
	2018-12-31 00:00:00, fhr = 6
		{'landn', 'vis'}
	2019-12-31 00:00:00, fhr = 0
		{'cfrzr', 'vis', 'prate', 'csnow', 'landn', 'crain', 'cicep'}
	2019-12-31 00:00:00, fhr = 6
		{'cfrzr', 'vis', 'prate', 'cpr', 'csnow', 'landn', 'crain', 'cicep'}
	2020-12-31 00:00:00, fhr = 0
		{'cfrzr', 'vis', 'prate', 'csnow', 'landn', 'crain', 'cicep'}
	2020-12-31 00:00:00, fhr = 6
		{'cfrzr', 'vis', 'prate', 'cpr', 'csnow', 'landn', 'crain', 'cicep'}
	2021-12-31 00:00:00, fhr = 0
		{'veg', 'fsr', 'cfrzr', 'vis', 'prate', 'slt', 'csnow', 'sit', 'crain', 'fricv', 'cicep'}
	2021-12-31 00:00:00, fhr = 6
		{'veg', 'fsr', 'cfrzr', 'vis', 'prate', 'cpr', 'slt', 'csnow', 'sit', 'crain', 'fricv', 'cicep'}
	2022-12-31 00:00:00, fhr = 0
		{'veg', 'fsr', 'cfrzr', 'vis', 'prate', 'slt', 'csnow', 'sit', 'crain', 'fricv', 'cicep'}
	2022-12-31 00:00:00, 

### Now, let's open a dataset, get these variables, and write out an updated dict

### First, instant stepType

Note that most of the time `cnwat` and `sithick` are only in `"b"`, but sometimes they are in both...
When they're in both, they appear to be exactly the same, since the dataset merges with no problem.

So... seems safe to set the file suffix for these to `"b"`

In [166]:
stepType = "instant"
itime = 0
ads = open_datasets(
    dims={"t0": gfs.t0[itime], "fhr": gfs.fhr[0]},
    cache_dir="./gribcache",
    filter_by_keys={
        "typeOfLevel": "surface",
        "stepType": stepType,
    },
)
fds = open_datasets(
    dims={"t0": gfs.t0[itime], "fhr": gfs.fhr[1]},
    cache_dir="./gribcache",
    filter_by_keys={
        "typeOfLevel": "surface",
        "stepType": stepType,
    },
)

ads = ads[sorted(intersect[0][stepType])]
fds = fds[sorted(intersect[6][stepType])]
xds = xr.concat([ads, fds], dim="fhr")

if "unknown" in xds:
    xds = xds.drop_vars("unknown")
if "t" in xds:
    xds = xds.rename({"t": "t_surface"})
    xds["t_surface"].attrs["original_name"] = "t"
    xds["t_surface"].attrs["long_name"] += " at surface"

dsdict["instant"] = xds.copy()

### Now accum and avg

Note that ... I don't really know what "avg" implies.
Is this averaged over the forecast lenth?
Over a 3hour period, or since the last diagnostic was output?

Some of the avg variables say "Time-mean" ... so I'll just copy that to the ones that don't

In [190]:
itime = 0
for stepType in ["avg", "accum"]:
    xds = open_datasets(
        dims={"t0": gfs.t0[itime], "fhr": gfs.fhr[1]},
        cache_dir="./gribcache",
        filter_by_keys={
            "typeOfLevel": "surface",
            "stepType": stepType,
        },
    )
    
    xds = xds[sorted(intersect[6][stepType])]
    if "unknown" in xds:
        xds = xds.drop_vars("unknown")
    if "t" in xds:
        xds = xds.rename({"t": "t_surface"})
        xds["t_surface"].attrs["original_name"] = "t"
        xds["t_surface"].attrs["long_name"] += " at surface"
    for varname in intersect[6][stepType]:
        if stepType not in varname:
            new = f"{stepType}_{varname}"
            xds = xds.rename({varname: new})
            xds[new].attrs["original_name"] = varname
            
            msg = "accumulated" if stepType == "accum" else "averaged"
            if stepType == "accum":
                xds[new].attrs["long_name"] += f" accumulated over forecast"
            else:
                xds[new].attrs["long_name"] =  "Time-mean " + xds[new].attrs["long_name"]
    
    dsdict[stepType] = xds

In [191]:
final = xr.merge(list(dsdict.values()))

In [192]:
final

<xarray.Dataset> Size: 349MB
Dimensions:      (t0: 1, latitude: 721, longitude: 1440, fhr: 2)
Coordinates:
  * t0           (t0) datetime64[ns] 8B 2015-12-31
  * latitude     (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude    (longitude) float64 12kB 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * fhr          (fhr) int64 16B 0 6
    lead_time    (fhr) timedelta64[ns] 16B 00:00:00 06:00:00
    surface      float64 8B 0.0
    valid_time   (t0, fhr) datetime64[ns] 16B 2015-12-31 2015-12-31T06:00:00
Data variables: (12/42)
    SUNSD        (t0, fhr, latitude, longitude) float32 8MB 0.0 0.0 ... 2.16e+04
    cape         (t0, fhr, latitude, longitude) float32 8MB 0.0 0.0 ... 0.0 0.0
    cin          (t0, fhr, latitude, longitude) float32 8MB 0.0 0.0 ... 0.0 0.0
    cnwat        (t0, fhr, latitude, longitude) float32 8MB nan nan ... 0.0 0.0
    cpofp        (t0, fhr, latitude, longitude) float32 8MB 100.0 100.0 ... 0.0
    fldcp        (t0, fhr, latitude, longitude) float32 8MB nan nan ... 0.2485
    ...           ...
    avg_sdswrf   (t0, fhr, latitude, longitude) float32 8MB nan nan ... 470.0
    avg_sulwrf   (t0, fhr, latitude, longitude) float32 8MB nan nan ... 195.0
    avg_suswrf   (t0, fhr, latitude, longitude) float32 8MB nan nan ... 395.0
    accum_acpcp  (t0, fhr, latitude, longitude) float32 8MB nan nan ... 0.0 0.0
    accum_tp     (t0, fhr, latitude, longitude) float32 8MB nan nan ... 0.0 0.0
    accum_watr   (t0, fhr, latitude, longitude) float32 8MB nan nan ... 0.0 0.0
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2025-06-09T10:29 GRIB to CDM+CF via cfgrib-0.9.1...

In [205]:
newdict = {}
for stepType, xds in dsdict.items():
    for varname in sorted(xds.data_vars):
        newdict[varname] = {
            "filter_by_keys": {
                "typeOfLevel": xds[varname].GRIB_typeOfLevel,
                "paramId": xds[varname].GRIB_paramId,
            },
            "long_name": xds[varname].long_name,
            "file_suffixes": xds[varname].file_suffix,
            "forecast_only": stepType != "instant",
        }
        if xds[varname].GRIB_typeOfLevel == "heightAboveGround":
            newdict[varname]["filter_by_keys"]["level"] = xds[varname].attrs["GRIB_level"]
        elif xds[varname].GRIB_typeOfLevel == "surface":
            newdict[varname]["filter_by_keys"]["stepType"] = xds[varname].attrs["GRIB_stepType"]
        if "original_name" in xds[varname].attrs:
            newdict[varname]["original_name"] = xds[varname].original_name

In [206]:
newdict = {key: newdict[key] for key in sorted(list(newdict.keys()))}

In [207]:
newdict

{'SUNSD': {'filter_by_keys': {'typeOfLevel': 'surface',
   'paramId': 7001292,
   'stepType': 'instant'},
  'long_name': 'Sunshine Duration',
  'file_suffixes': [''],
  'forecast_only': False},
 'accum_acpcp': {'filter_by_keys': {'typeOfLevel': 'surface',
   'paramId': 3063,
   'stepType': 'accum'},
  'long_name': 'Convective precipitation (water) accumulated over forecast',
  'file_suffixes': [''],
  'forecast_only': True,
  'original_name': 'acpcp'},
 'accum_tp': {'filter_by_keys': {'typeOfLevel': 'surface',
   'paramId': 228228,
   'stepType': 'accum'},
  'long_name': 'Total Precipitation accumulated over forecast',
  'file_suffixes': [''],
  'forecast_only': True,
  'original_name': 'tp'},
 'accum_watr': {'filter_by_keys': {'typeOfLevel': 'surface',
   'paramId': 260181,
   'stepType': 'accum'},
  'long_name': 'Water runoff accumulated over forecast',
  'file_suffixes': [''],
  'forecast_only': True,
  'original_name': 'watr'},
 'avg_al': {'filter_by_keys': {'typeOfLevel': 'surface

In [208]:
import yaml

In [209]:
sources.__path__[0]

'/Users/tsmith/work/ufs2arco/ufs2arco/sources'

In [210]:
with open(f"{sources.__path__[0]}/reference.gfs.yaml", "r") as f:
    reference = yaml.safe_load(f)

In [211]:
updated = reference.copy()

In [212]:
updated.update(newdict)


In [213]:
updated["lsm"]

{'filter_by_keys': {'typeOfLevel': 'surface',
  'paramId': 172,
  'stepType': 'instant'},
 'long_name': 'Land-sea mask',
 'file_suffixes': [''],
 'forecast_only': False}

In [214]:
reference["lsm"]

{'file_suffixes': [''],
 'filter_by_keys': {'paramId': 172, 'typeOfLevel': 'surface'},
 'forecast_only': False,
 'long_name': 'Land-sea mask'}

In [215]:
updated = {key: updated[key] for key in sorted(updated.keys())}

In [216]:
with open("reference.gfs.yaml", "w") as f:
    yaml.dump(updated, f)